# Vulnerability Assessment Pilot
This notebook demonstrates on-going development of climate adaptation vulnerability assessment (CAVA) support using climate data in the Analytics Engine. 

To execute a given 'cell' of this notebook, place the cursor in the cell and press the 'play' icon, or simply press shift+enter together. Some cells will take longer to run, and you will see a [$\ast$] to the left of the cell while AE is still working.

**Intended Application**: As a user, I want to **<span style="color:#FF0000">access climate projections data for my vulnerability assessment report</span>** by:
1. Retrieve data metrics required for planning needs

**Runtime**: With the default settings, this notebook takes approximately **several hours** to run from start to finish, depending on the metric choice. Modifications to selections may increase the runtime. 

### Step 0: Set-up

First, we'll import the Python library [climakitae](https://github.com/cal-adapt/climakitae), our AE toolkit for climate data analysis, along with this specific functions from that library that we'll use in this notebook, as well as any other necessary Python libraries to aid in analysis.

In [4]:
from climakitae.explore.vulnerability import cava_data
from climakitae.explore.vulnerability_table import create_vul_table

### Step 1: Import locations
Now we'll read in point-based locations that we want to retrieve data for. For custom inputs, there are two options: (1) Input a single pair of latitude - longitude values; and (2) Import a csv file of locations that will each run. In the code below we show what each option looks like. 

To import your own custom locations, we recommend putting your csv file in the same folder as this notebook for ease:
1. Drag and drop a csv file into the file tree on the left hand side; or
2. Use the `upload` button (the "up arrow" symbol next to the large blue plus symbol above the file tree). 

<span style="color:#FF0000">**Formatting note**</span>: For the code cells below to work, there must be **2 columns labeled `lat` and `lon`**. Functionality to accept different labeling is forthcoming!

In the cell below, we read the csv file in. We use the HadISD station list as an example here -- you may want to replace with your own locations file!

In [ ]:
import pandas as pd
locations = pd.read_csv('*.csv')

### Step 2: Retrieve metric data

The `cava_data` funciontality is designed to provide flexibility over customizable metric calculation. There are 4 customizable metrics that can be built with this functionality:
1. Likely seasonal event occurence (e.g., "likely summer night low temperature")
2. 1-in-X temperature events (e.g., "1-in-10 year maximum temperature")
3. High/Extreme Heat Index events (e.g., "how many days per year does the Heat Index exceed 90°F")
4. 1-in-X precipitation events (e.g., "1-in-100 year, 24 hour precipitation")

Below is a table outlining all avaialble arguments to the `cava_data` function. The "Required" flag notes whether the argument must be passed to start generating data. Input options for each argument is provided, as well as whether a setting is required for any of the required selections. We provide multiple examples of working with the `cava_data` function with multiple configurations.

| Argument | Options | Argument required for | Notes |
|----------|---------|-----------------------|-------|
|input_locations | Pass a location via csv. | All | Option to run either a single location, or multiple when **batch_mode=True**.|
|variable | "Air Temperature at 2m", "NOAA Heat Index", "Precipitation (total)"| All | |
|approach | "Time", "Warming Level" | All | |
|downscaling_method | "Dynamical", "Statistical"| All | |
|time_start_year | Numerical (min is 1981) | Required for **approach=Time** | |
|time_end_year | Numerical (max is 2100) | Required for **approach=Time**| |
|historical_data| "Historical Climate", "Historical Reconstruction" | Required for **approach=Time**| **Historical Climate** ranges from 1980-2015 for WRF and 1950-2015 for LOCA2-Hybrid. **Historical Reconstruction** ranges from 1950-2022. Historical Reconstruction data cannot be combined with SSP data.|
|ssp_data | "[SSP2-4.5]", "[SSP3-7.0]", "[SSP5-8.5]" | Required for **approach=Time** | Dynamical only has SSP3-7.0, Statisical has all 3 SSP options|
|warming_level| 1.5, 2.0, 2.5, 3.0 | Required for **approach=Warming Level**| |
|metric_calc| "max", "min" | Required for 1-in-X events, Heat Index, likely seasonal event | |
|heat_idx_threshold | Numerical | Required for Heat Index | Heat Index can only be calculated with **downscaling_method="Dynamical"**|
|one_in_x | Numerical | Required for 1-in-X events| |
|event_duration| Numerical + "day"/"hour"| Optional for 1-in-X events| Must adhere to following structure: ({number}, "{temporal frequency}"). Temporal frequency options: "day", "hour". Default is (1, "day").|
|distr| "gev", "genpareto", "gumbel", "wibull", "pearson3", "gamma"| Optional for 1-in-X events |Default set to "gev". |
|percentile | Numerical (0-100) | Required for likely seasonal event |   |
|season| "summer", "winter", "all" | Required for likely seasonal event| Default set to "all"|
|units | Temp/Heat Index: "degF", "degC", "K". Precip: "mm", "inches" | Optional | Default for temp/Heat Index is DegF. Default for precip is mm.|
|wrf_bias_adjust| True, False | Optional| Option to select only the 4 bias-adjusted WRF models. Only applicable for **downscaling_method="Dynamical"**. Default set to True |
|export_method| "raw", "calculate", "both" | Optional | Default set to "both" |
|file_format | "NetCDF", "csv" | Optional | Default set to "NetCDF" |
|separate_files | True, False | Optional | Option to export separate files if multiple points are passed. Default set to True|
|batch_mode | True, False | Optional, but recommmended for multiple locations. | Option to efficiently run multiple points. Separate files for export is turned off in batch mode. Default set to False. |

The following cells illustrate several examples of how to retrieve and calculate various configurations of the `cava_data` function. Below is a list of the examples; and more are coming soon as more functionality is built in!
1. Likely seasonal event, single location, time approach, all WRF data, with custom percentile
2. Likely seasonal event, batch mode for multiple locations, time approach, all WRF data, with custom percentile
3. 1-in-X temperature event, single location, bias-adjusted WRF data only, time approach, with custom event frequency and distribution
4. Heat index event, single location, time approach, with custom threshold
5. Likely seasonal event, single location, warming level approach, all WRF data, wtih custom percentile
6. Likely seasonal event, single location, warming level approach, all LOCA2 data, with custom percentile
7. Heat index event, batch mode for multiple locations, time approach, Historical Reconstruction data, with custom threshold
8. Likely seasonal event, batch mode for multiple locations, warming levels approach, all LOCA2 data, with custom percentile
9. 1-in-X precipitation event, single location, all LOCA2 data, time approach, with custom return period
10. 1-in-X precipitation event, single location, all WRF data, time approach, with custom return period, event duration, and distribution
11. Example of reading the calculated metric data via xarray for easy viewing within this notebook. 


# 2.0C Warming Scenario
!chmod +x upload_nc_to_gcs.sh
# Loop through each location in example_locs
for pt in range(len(locations)):
    data = cava_data(
        ## Set-up
        example_locs[pt:(pt+1)],
        downscaling_method="Statistical",  # LOCA2 data 
        approach="Warming Level",  
        warming_level=1.0,
        
        ## 1-in-X event specific arguments
        variable="Precipitation (total)",
        metric_calc="max", # daily maximum precipitation
        one_in_x=100, # One-in-X
        distr="gev", # change distribution
        units="inches", # change units
        
        ## Export
        export_method="calculate",
        file_format="NetCDF"
        #batch_mode=True # batch mode for GWL LOCA still in development
    )
    ## Add function to upload all nc files to GCP bucket if mannually initiate the batch run from here instead of from cava-orchestrator
    !./upload_nc_to_gcs.sh

   

In [ ]:

# 2.0C Warming Scenario
!chmod +x upload_nc_to_gcs_2WL.sh
# Loop through each location in example_locs
for pt in range(len(example_locs)):
    data = cava_data(
        ## Set-up
        example_locs[pt:(pt+1)],
        downscaling_method="Statistical",  # LOCA2 data 
        approach="Warming Level",  
        warming_level=2.0,
        
        ## 1-in-X event specific arguments
        variable="Precipitation (total)",
        metric_calc="max", # daily maximum precipitation
        one_in_x=100, # One-in-X
        distr="gev", # change distribution
        units="inches", # change units
        
        ## Export
        export_method="calculate",
        file_format="NetCDF"
        #batch_mode=True # batch mode for GWL LOCA still in development
    )
    ## Add function to upload all nc files to GCP bucket if mannually initiate the batch run from here instead of from cava-orchestrator
    !./upload_nc_to_gcs_2WL.sh

    # 1.5C Warming Scenario
!chmod +x upload_nc_to_gcs_15WL.sh
for pt in range(len(wl15_locs)):
    data = cava_data(
        ## Set-up
        wl15_locs[pt:(pt+1)],
        downscaling_method="Statistical",  # LOCA2 data 
        approach="Warming Level",  
        warming_level=1.0,
        
        ## 1-in-X event specific arguments
        variable="Precipitation (total)",
        metric_calc="max", # daily maximum precipitation
        one_in_x=100, # One-in-X
        distr="gev", # change distribution
        units="inches", # change units
        
        ## Export
        export_method="calculate",
        file_format="NetCDF"
        #batch_mode=True # batch mode for GWL LOCA still in development
    )
    ## Add function to upload all nc files to GCP bucket if mannually initiate the batch run from here instead of from cava-orchestrator

    !./upload_nc_to_gcs_15WL.sh

#### Example: Looking at the `cava_data` output
It may be useful to look at the `cava_data` output within this notebook to assess the results and make any changes to the data request. After running the `cava_data` function, in a new cell you can type `data` to view the xarray data object. Depending on your export setting ("raw", "calculate", "both"), you can also view the data object in a more user-friendly xarray view. We provide the code to do so in the next cell -- select which option matches your `cava_data` run and the export option you would like to view! 

In [ ]:
locagrids_lab20.csvdata # looking at the full xarray data object; will be a dictionary of data arrays!
# data['calc_data'] # looking at just the calculated data metric
# data['raw'] # looking at just the raw input data

---

### Appendix: Table Generation Sample Code

Below, you'll find code that generates a table with different climate data metrics used in a CAVA Report. Feel free to run it and check it out! It is still very much in progress. **This will take 30+ min. to run.**

In [ ]:
%%time
percentile = 50
heat_idx_threshold = 80
one_in_x = 10 # currently, only can do `one_in_x` for one value at a time
df = create_vul_table(example_locs.iloc[[10]], percentile, heat_idx_threshold, one_in_x)
df